**Install required Packages**

Load in dublin_merged_df.csv.gz and svd_final_model.pkl

In [34]:
!pip install vaderSentiment
!pip install scikit-surprise
!pip install folium
!pip install surprise

In [35]:
!pip install "numpy<2.0"

In [36]:
# Basic libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud
import re
import warnings

# Advanced ML
import xgboost as xgb
import lightgbm as lgb
from lightgbm import LGBMRegressor

# Sentiment
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

# Recommendation system
from surprise import Reader, Dataset, SVD, accuracy, BaselineOnly, SlopeOne
from surprise.model_selection import train_test_split, GridSearchCV, cross_validate

# Utilities
import joblib
import random

# Plotting config
warnings.filterwarnings("ignore")
warnings.simplefilter(action='ignore', category=FutureWarning)

In [37]:
# load listings
dublin_merged_df = pd.read_csv('dublin_merged_df.csv.gz', compression='gzip')
dublin_merged_df.head(1)

,id_x,listing_url,name,description,neighborhood_overview,picture_url,host_id,host_response_time,host_response_rate,host_is_superhost,host_listings_count,host_total_listings_count,host_verifications,host_identity_verified,neighbourhood,neighbourhood_cleansed,latitude,longitude,property_type,room_type,accommodates,bathrooms,bathrooms_text,bedrooms,beds,amenities,price,number_of_reviews,availability_eoy,number_of_reviews_ly,estimated_occupancy_l365d,estimated_revenue_l365d,instant_bookable,date,reviewer_id,comments,avg_rating,compound_scores,month,season
0,44077,https://www.airbnb.com/rooms/44077,cosy comfortable Irish cottage twin,Our house is a cosy comfortable cottage with ...,I like our neighbourhood as there is no shorta...,https://a0.muscache.com/pictures/525706/050a3a...,193005,within a few hours,100%,t,2.0,2.0,"['email', 'phone']",t,"Churchtown, Ireland",Dn Laoghaire-Rathdown,53.29178,-6.25792,Private room in bungalow,Private room,2,1.0,1 private bath,1.0,2.0,"[""Shared patio or balcony"", ""Refrigerator"", ""C...",76.0,380,42,42,234,17784.0,f,2011-03-20,393348,We enjoyed our stay very much. The room was co...,4.827143,0.9849,3,Spring


In [38]:
loaded_model = joblib.load('svd_final_model.pkl')

In [39]:
def recommend_airbnbs_gradio(user_id):
    import pandas as pd

    user_id = int(user_id)

    # Filter reviews by this user
    user_reviews = dublin_merged_df[dublin_merged_df['reviewer_id'] == user_id]
    user_listings = user_reviews['id_x'].unique()

    # All listings
    all_listings = dublin_merged_df['id_x'].unique()
    listings_to_predict = list(set(all_listings) - set(user_listings))

    if len(listings_to_predict) == 0:
        return "<p>No new listings to recommend for this user.</p>"

    # Predict ratings
    user_listing_pairs = [(user_id, listing_id, 0) for listing_id in listings_to_predict]
    predictions = loaded_model.test(user_listing_pairs)

    # Top 5 predictions
    top_5_recs = sorted(predictions, key=lambda x: x.est, reverse=True)[:5]

    # Build HTML
    html_output = []

    for idx, rec in enumerate(top_5_recs):
        listing_row = dublin_merged_df[dublin_merged_df['id_x'] == rec.iid]

        if listing_row.empty:
            continue

        row = listing_row.iloc[0]

        description = str(row.get("description", "No description"))
        url = row.get("listing_url", "#")
        image_url = row.get("picture_url", "")

        card = f"""
        <div style="margin-bottom: 20px;">
            <h4>Recommendation #{idx + 1}</h4>
            <a href="{url}" target="_blank">{description}</a><br>
            <img src="{image_url}" alt="Listing Image" width="300"><br>
        </div>
        """
        html_output.append(card)

    if not html_output:
        return "<p>No valid recommendations could be generated.</p>"

    return ''.join(html_output)

In [40]:
import gradio as gr

iface = gr.Interface(
    fn=recommend_airbnbs_gradio,
    inputs=gr.Number(label="Enter Your Reviewer ID"),
    outputs=gr.HTML(label="Recommended Listings"),
    title="Dublin Airbnb Recommendation Engine",
    description="Enter your reviewer ID to get 5 personalized Airbnb listings in Dublin!"
)

iface.launch()

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://86c2dbdde905978d81.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [33]:
dublin_merged_df.head(10)

,id_x,listing_url,name,description,neighborhood_overview,picture_url,host_id,host_response_time,host_response_rate,host_is_superhost,host_listings_count,host_total_listings_count,host_verifications,host_identity_verified,neighbourhood,neighbourhood_cleansed,latitude,longitude,property_type,room_type,accommodates,bathrooms,bathrooms_text,bedrooms,beds,amenities,price,number_of_reviews,availability_eoy,number_of_reviews_ly,estimated_occupancy_l365d,estimated_revenue_l365d,instant_bookable,date,reviewer_id,comments,avg_rating,compound_scores,month,season
0,44077,https://www.airbnb.com/rooms/44077,cosy comfortable Irish cottage twin,Our house is a cosy comfortable cottage with ...,I like our neighbourhood as there is no shorta...,https://a0.muscache.com/pictures/525706/050a3a...,193005,within a few hours,100%,t,2.0,2.0,"['email', 'phone']",t,"Churchtown, Ireland",Dn Laoghaire-Rathdown,53.29178,-6.25792,Private room in bungalow,Private room,2,1.0,1 private bath,1.0,2.0,"[""Shared patio or balcony"", ""Refrigerator"", ""C...",76.0,380,42,42,234,17784.0,f,2011-03-20,393348,We enjoyed our stay very much. The room was co...,4.827143,0.9849,3,Spring
1,44077,https://www.airbnb.com/rooms/44077,cosy comfortable Irish cottage twin,Our house is a cosy comfortable cottage with ...,I like our neighbourhood as there is no shorta...,https://a0.muscache.com/pictures/525706/050a3a...,193005,within a few hours,100%,t,2.0,2.0,"['email', 'phone']",t,"Churchtown, Ireland",Dn Laoghaire-Rathdown,53.29178,-6.25792,Private room in bungalow,Private room,2,1.0,1 private bath,1.0,2.0,"[""Shared patio or balcony"", ""Refrigerator"", ""C...",76.0,380,42,42,234,17784.0,f,2011-03-28,444004,We have been here 4 nights. Stay in a home is ...,4.827143,0.9846,3,Spring
2,44077,https://www.airbnb.com/rooms/44077,cosy comfortable Irish cottage twin,Our house is a cosy comfortable cottage with ...,I like our neighbourhood as there is no shorta...,https://a0.muscache.com/pictures/525706/050a3a...,193005,within a few hours,100%,t,2.0,2.0,"['email', 'phone']",t,"Churchtown, Ireland",Dn Laoghaire-Rathdown,53.29178,-6.25792,Private room in bungalow,Private room,2,1.0,1 private bath,1.0,2.0,"[""Shared patio or balcony"", ""Refrigerator"", ""C...",76.0,380,42,42,234,17784.0,f,2011-04-21,465058,Teresa and Hughie were great hosts. They were ...,4.827143,0.9836,4,Spring
3,44077,https://www.airbnb.com/rooms/44077,cosy comfortable Irish cottage twin,Our house is a cosy comfortable cottage with ...,I like our neighbourhood as there is no shorta...,https://a0.muscache.com/pictures/525706/050a3a...,193005,within a few hours,100%,t,2.0,2.0,"['email', 'phone']",t,"Churchtown, Ireland",Dn Laoghaire-Rathdown,53.29178,-6.25792,Private room in bungalow,Private room,2,1.0,1 private bath,1.0,2.0,"[""Shared patio or balcony"", ""Refrigerator"", ""C...",76.0,380,42,42,234,17784.0,f,2011-05-13,490005,"No surprises, was as described. Very gracious...",4.827143,0.9412,5,Spring
4,44077,https://www.airbnb.com/rooms/44077,cosy comfortable Irish cottage twin,Our house is a cosy comfortable cottage with ...,I like our neighbourhood as there is no shorta...,https://a0.muscache.com/pictures/525706/050a3a...,193005,within a few hours,100%,t,2.0,2.0,"['email', 'phone']",t,"Churchtown, Ireland",Dn Laoghaire-Rathdown,53.29178,-6.25792,Private room in bungalow,Private room,2,1.0,1 private bath,1.0,2.0,"[""Shared patio or balcony"", ""Refrigerator"", ""C...",76.0,380,42,42,234,17784.0,f,2011-05-17,520460,"Teresa was a lovely hostess, and we had a deli...",4.827143,0.9835,5,Spring
5,44077,https://www.airbnb.com/rooms/44077,cosy comfortable Irish cottage twin,Our house is a cosy comfortable cottage with ...,I like our neighbourhood as there is no shorta...,https://a0.muscache.com/pictures/525706/050a3a...,193005,within a few hours,100%,t,2.0,2.0,"['email', 'phone']",t,"Churchtown, Ireland",Dn Laoghaire-Rathdown,53.29178,-6.25792,Private room in bungalow,Private room,2,1.0,1 private bath,1.0,2.0,"[""Shared patio o